# Genetic Algorithm

In this project, the aim is to solve a sudoku puzzle using the genetic algorithm

In [1]:
import random as rndm
import time

## Part 1: Defining Genes and Chromosomes

A gene is the value of a row and is a permutation of set {1 ... 9}. A chromosome consists of 9 genes, each gene representing a row of the actual sudoku puzzle.

In [2]:
def make_gene(initial=None):
    if initial is None:
        initial = [0] * 9
    mapp = {}
    gene = list(range(1, 10))
    rndm.shuffle(gene)
    for i in range(9):
        mapp[gene[i]] = i
    for i in range(9):
        if initial[i] != 0 and gene[i] != initial[i]:
            temp = gene[i], gene[mapp[initial[i]]]
            gene[mapp[initial[i]]], gene[i] = temp
            mapp[initial[i]], mapp[temp[0]] = i, mapp[initial[i]]
    return gene

In [3]:
def make_chromosome(initial=None):
    if initial is None:
        initial = [[0] * 9] * 9
    chromosome = []
    for i in range(9):
        chromosome.append(make_gene(initial[i]))
    return chromosome

## Part 2: Making First Generation

In this part, a function is implemented to randomly create a population.

In [4]:
def make_population(count, initial=None):
    if initial is None:
        initial = [[0] * 9] * 9
    population = []
    for _ in range(count):
        population.append(make_chromosome(initial))
    return population

## Part 3: Fitness Function

The fitness function calculates how "fit" a chromosome (puzzle) is based on:
- For each column: Subtract (number of times a number is seen) - 1 from the fitness for that number
- For each 3x3 square: Subtract (number of times a number is seen) - 1 from the fitness for that number
The higher the fitness, the closer the puzzle is to being solved.

In [15]:
def get_fitness(chromosome):
    """Calculate the fitness of a chromosome (Sudoku puzzle)."""
    fitness = 0

    # Check rows
    for row in chromosome:
        unique_numbers = set(row)
        fitness += len(unique_numbers)  # Reward for unique numbers

    # Check columns
    for col in range(9):
        seen = set()
        for row in range(9):
            seen.add(chromosome[row][col])
        fitness += len(seen)  # Reward for unique numbers in column

    # Check 3x3 sub-grids
    for grid_row in range(0, 9, 3):
        for grid_col in range(0, 9, 3):
            seen = set()
            for row in range(grid_row, grid_row + 3):
                for col in range(grid_col, grid_col + 3):
                    seen.add(chromosome[row][col])
            fitness += len(seen)  # Reward for unique numbers in grid

    # Max fitness is 243 (81 per rows + columns + grids)
    # Scale to return penalties for missing constraints
    return fitness - 243


In [6]:
ch = make_chromosome()
print(get_fitness(ch))


def pch(ch):
    for i in range(9):
        for j in range(9):
            print(ch[i][j], end=" ")
        print("")


-59


## Part 4: Crossover and Mutation

In this part, the crossover and mutation function is implemented to help in determining the next generation.

### Crossover

the crossover function takes two chromosomes as input and makes two new chromosomes by combining them. This crossover function decides the parent of each gene separately, so the result is independent of the location of the genes.

In [7]:
def crossover(ch1, ch2):
    new_child_1 = []
    new_child_2 = []
    for i in range(9):
        x = rndm.randint(0, 1)
        if x == 1:
            new_child_1.append(ch1[i])
            new_child_2.append(ch2[i])
        elif x == 0:
            new_child_2.append(ch1[i])
            new_child_1.append(ch2[i])
    return new_child_1, new_child_2

### Mutation

In [8]:
def mutation(ch, pm, initial):
    for i in range(9):
        x = rndm.randint(0, 100)
        if x < pm * 100:
            ch[i] = make_gene(initial[i])
    return ch

## Part 5: Implementing The Genetic Algorithm

In this part, we use the components defined in previous steps to write the genetic algorithm.

Read the puzzle from the given address


In [9]:
def read_puzzle(address):
    puzzle = []
    f = open(address, 'r')
    for row in f:
        temp = row.split()
        puzzle.append([int(c) for c in temp])
    return puzzle

Get the mating pool using the random method

In [10]:
def r_get_mating_pool(population):
    fitness_list = []
    pool = []
    for chromosome in population:
        fitness = get_fitness(chromosome)
        fitness_list.append((fitness, chromosome))
    fitness_list.sort()
    weight = list(range(1, len(fitness_list) + 1))
    for _ in range(len(population)):
        ch = rndm.choices(fitness_list, weight)[0]
        pool.append(ch[1])
    return pool


Get the mating pool using the weighted method

In [11]:
def w_get_mating_pool(population):
    fitness_list = []
    pool = []
    for chromosome in population:
        fitness = get_fitness(chromosome)
        fitness_list.append((fitness, chromosome))
    weight = [fit[0] - fitness_list[0][0] for fit in fitness_list]
    for _ in range(len(population)):
        ch = rndm.choices(fitness_list, weights=weight)[0]
        pool.append(ch[1])
    return pool


Generate the offsprings from the mating pool

In [12]:
def get_offsprings(population, initial, pm, pc):
    new_pool = []
    i = 0
    while i < len(population):
        ch1 = population[i]
        ch2 = population[(i + 1) % len(population)]
        x = rndm.randint(0, 100)
        if x < pc * 100:
            ch1, ch2 = crossover(ch1, ch2)
        new_pool.append(mutation(ch1, pm, initial))
        new_pool.append(mutation(ch2, pm, initial))
        i += 2
    return new_pool

In [13]:
# Population size
POPULATION = 1000

# Number of generations
REPETITION = 1000

# Probability of mutation
PM = 0.1

# Probability of crossover
PC = 0.95

# Main genetic algorithm function
def genetic_algorithm(initial_file):
    initial = read_puzzle(initial_file)
    population = make_population(POPULATION, initial)
    for _ in range(REPETITION):
        mating_pool = r_get_mating_pool(population)
        rndm.shuffle(mating_pool)
        population = get_offsprings(mating_pool, initial, PM, PC)
        fit = [get_fitness(c) for c in population]
        m = max(fit)
        if m == 0:
            return population
    return population

Run the algorithm and time it

In [18]:
tic = time.time()
r = genetic_algorithm("./sample_sudoku/Test2.txt")
toc = time.time()
print("time_taken: ", toc - tic)
fit = [get_fitness(c) for c in r]
m = max(fit)
print('max fitness :', max(fit))

# Print the chromosome with the highest fitness
for c in r:
    if get_fitness(c) == m:
        pch(c)
        break

time_taken:  73.37765145301819
max fitness : 0
9 6 8 2 5 3 4 7 1 
4 7 5 1 9 6 3 8 2 
3 1 2 4 8 7 6 9 5 
2 5 1 9 4 8 7 6 3 
7 9 3 6 2 5 8 1 4 
8 4 6 3 7 1 2 5 9 
1 8 7 5 3 2 9 4 6 
6 2 9 8 1 4 5 3 7 
5 3 4 7 6 9 1 2 8 


## Problems with Genetic Algorithm

The algorithm may encounter a challenge when it becomes trapped at a local maximum and the desired solution, such as in a Sudoku problem, cannot be achieved. One approach to address this issue is to initially increase the mutation rate, although this can have its own drawbacks as mentioned earlier. Alternatively, a function can be defined to identify local maximums, such as by detecting repeated values, and in such cases, the mutation rate can be increased. Another solution involves increasing the population size or the number of iterations the algorithm runs. Modifying the fitness function or changing the method for selecting individuals in the mating pool are additional strategies. Overall, since the algorithm's nature involves randomness, it is possible to encounter this problem regardless of the chosen method, but running the algorithm multiple times can sometimes be helpful.